In [15]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, concatenate , LSTM
from keras import Model
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix , f1_score
from keras.callbacks import EarlyStopping
import xgboost as xgb
from xgboost import XGBClassifier
from openpyxl import load_workbook
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

In [16]:
def apply_proba_threshold(y_positive_proba_pred, threshold):
    apply_threshold = np.vectorize(lambda x:0 if x< threshold else 1)
    return apply_threshold(y_positive_proba_pred)
def optimize_binary_threshold(y_proba_pred, y_true, thresholds, metric = 'f1'):
    best_score = 0
    best_thresh = 0.1
    
    for thresh in thresholds : 
        y_pred = apply_proba_threshold(y_proba_pred, thresh)
        
        score = f1_score(y_pred = y_pred,y_true = y_true)
        
            
        if score > best_score:
            best_score = score
            best_thresh = thresh
    
    return best_thresh

In [17]:
### embedding ###
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
print('embedding chargé')

embedding chargé


In [18]:
### lecture des données ####
train = pd.read_csv("Data/train.csv")
test = pd.read_csv("Data/test.csv")

In [19]:
xtrain = train[['text','keyword','location']]
ytrain = train['target']
xtest = test[['text','keyword','location']]

In [20]:
def no_at(text):
    p = re.compile(r'\s')
    l = p.split(text)
    for mot in l :
        if '@' in mot :
            l.remove(mot)
    text_final = ''
    for mot in l:
        text_final +=mot
        text_final += ' '
    return text_final
def text_traitement(text_init):
    text_init = no_at(text_init)
    text_final = ''
    for carac in text_init:
        print("no")
        
        
        

In [21]:
for i in range(2):
    xtrain['text'] = xtrain['text'].apply(no_at)
    xtest['text'] = xtest['text'].apply(no_at)

<ipython-input-21-a4ef583013c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain['text'] = xtrain['text'].apply(no_at)
<ipython-input-21-a4ef583013c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest['text'] = xtest['text'].apply(no_at)


In [22]:
print(xtrain)

                                                   text keyword location
0     Our Deeds are the Reason of this #earthquake M...     NaN      NaN
1              Forest fire near La Ronge Sask. Canada       NaN      NaN
2     All residents asked to 'shelter in place' are ...     NaN      NaN
3     13,000 people receive #wildfires evacuation or...     NaN      NaN
4     Just got sent this photo from Ruby #Alaska as ...     NaN      NaN
...                                                 ...     ...      ...
7608  Two giant cranes holding a bridge collapse int...     NaN      NaN
7609  The out of control wild fires in California ev...     NaN      NaN
7610  M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...     NaN      NaN
7611  Police investigating after an e-bike collided ...     NaN      NaN
7612  The Latest: More Homes Razed by Northern Calif...     NaN      NaN

[7613 rows x 3 columns]


In [23]:
dict_keyword = {}
j=0
for i in set(train['keyword']):
    dict_keyword[i] = j
    j+=1
dict_location = {}
j=0
for i in set(train['location']):
    dict_location[i] = j
    j+=1

In [24]:
### caractéristiques des données ###
print("Colonnes : ",list(train.columns))
print("Nombre de lignes du set de train : ", len(train))
print("Nombre de lignes du set de test : ", len(test))
print("Les classes sont : ",set(train['target']))

Colonnes :  ['id', 'keyword', 'location', 'text', 'target']
Nombre de lignes du set de train :  7613
Nombre de lignes du set de test :  3263
Les classes sont :  {0, 1}


In [25]:
### Split train into train and validation ###
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size = 0.3)
xtrain_text = np.array(embed(xtrain['text']))
xtrain_location = np.array([dict_location[i] for i in xtrain['location']])
xtrain_keyword = np.array([dict_keyword[i] for i in xtrain['keyword']])

xvalid_text = np.array(embed(xvalid['text']))
xvalid_location = np.array([dict_location[i] for i in xvalid['location']])
xvalid_keyword = np.array([dict_keyword[i] for i in xvalid['keyword']]) 

xtest_text = np.array(embed(xtest['text']))
xtest_location = []
xtest_keyword = []

for i in xtest['location']:
    if i in dict_location:
        xtest_location.append(dict_location[i])
    else:
        xtest_location.append(0)

for i in xtest['keyword']:
    if i in dict_keyword:
        xtest_keyword.append(dict_keyword[i])
    else:
        xtest_keyword.append(0)
        
xtest_location = np.array(xtest_location)
xtest_keyword = np.array(xtest_keyword)

In [26]:
xtrain_xgb = []
for i in range(len(xtrain_text)):
    m = list(xtrain_text[i])
    m.append(xtrain_location[i])
    m.append(xtrain_keyword[i])
    xtrain_xgb.append(m)
xtrain_xgb=np.array(xtrain_xgb)

xvalid_xgb = []
for i in range(len(xvalid_text)):
    m = list(xvalid_text[i])
    m.append(xvalid_location[i])
    m.append(xvalid_keyword[i])
    xvalid_xgb.append(m)
xvalid_xgb=np.array(xvalid_xgb)

In [63]:
# random forest 
clf = RandomForestClassifier(n_estimators = 350,max_depth=20, random_state=70)
clf.fit(xtrain_xgb, ytrain)
y_pred = clf.predict(xvalid_xgb)
CM = confusion_matrix(y_pred = y_pred,y_true = yvalid)
f1 = f1_score(y_pred = y_pred,y_true = yvalid)
print(CM)
print(f1)
# print(clf.predict_proba(xvalid_xgb))

[[1214  103]
 [ 307  660]]
0.7630057803468208


In [64]:
### xgboost model learning####
model = XGBClassifier(learning_rate = 0.15, n_estimators=400, max_depth=10,gamma=0.05)
model.fit(xtrain_xgb, ytrain)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.05, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.15, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=400, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [65]:
### xgboost model predict####
y_pred = model.predict(xvalid_xgb)
CM = confusion_matrix(y_pred = y_pred,y_true = yvalid)
f1 = f1_score(y_pred = y_pred,y_true = yvalid)
print(CM)
print(f1)

[[1182  135]
 [ 273  694]]
0.7728285077951002


In [27]:
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=20)
cb_list=[es]
def b():
    inputs_text = Input(shape=(512,))
    
    inputs_keyword = Input(shape=(1,))
    
    #inputs_location = Input(shape=(1,))

    
    
    first_layer_text = Dense(60, activation='sigmoid')(inputs_text)
    second_layer_text = Dense(30, activation='softplus')(first_layer_text)
    
    first_layer_keyword = Dense(1, activation='sigmoid')(inputs_keyword)
    second_layer_keyword = Dense(1, activation='softsign')(first_layer_keyword)
    
    #first_layer_location = Dense(1, activation='sigmoid')(inputs_location)
    #second_layer_location = Dense(1, activation='sigmoid')(first_layer_location)
    
    
    pre_out = concatenate([second_layer_text,second_layer_keyword])
    
    pre_out_out = Dense(30, activation='sigmoid')(pre_out)
    
    out = Dense(1,activation='sigmoid')(pre_out_out)
    
    loss_f = 'binary_crossentropy'
    
    model = Model(inputs=[inputs_text,inputs_keyword], outputs=[out])
    
    model.compile(optimizer='adam',loss=[loss_f],metrics=['accuracy'])
    
    model.fit([xtrain_text,xtrain_keyword],ytrain, epochs=200,batch_size=128,callbacks=cb_list,validation_split=0.1)
    
    return model

In [28]:
model = b()
ypred = model.predict([xvalid_text,xvalid_keyword])


Epoch 1/200
38/38 [==============================] - 0s 9ms/step - loss: 0.7479 - accuracy: 0.4912 - val_loss: 0.6961 - val_accuracy: 0.5478
Epoch 2/200
38/38 [==============================] - 0s 4ms/step - loss: 0.6707 - accuracy: 0.5811 - val_loss: 0.6745 - val_accuracy: 0.5478
Epoch 3/200
38/38 [==============================] - 0s 2ms/step - loss: 0.6548 - accuracy: 0.5811 - val_loss: 0.6542 - val_accuracy: 0.5535
Epoch 4/200
38/38 [==============================] - 0s 2ms/step - loss: 0.6235 - accuracy: 0.6293 - val_loss: 0.6103 - val_accuracy: 0.7242
Epoch 5/200
38/38 [==============================] - 0s 3ms/step - loss: 0.5561 - accuracy: 0.7923 - val_loss: 0.5326 - val_accuracy: 0.7617
Epoch 6/200
38/38 [==============================] - 0s 2ms/step - loss: 0.4742 - accuracy: 0.8036 - val_loss: 0.4700 - val_accuracy: 0.7861
Epoch 7/200
38/38 [==============================] - 0s 2ms/step - loss: 0.4303 - accuracy: 0.8130 - val_loss: 0.4512 - val_accuracy: 0.7824
Epoch 8/200
3

In [29]:
thresholds = [0.1+i*0.001 for i in range(900)]
print(max(thresholds))
thresh = optimize_binary_threshold(ypred, yvalid, thresholds, metric = 'f1')
print(thresh)
y_pred = apply_proba_threshold(ypred, thresh)
CM = confusion_matrix(y_pred = y_pred,y_true = yvalid)
f1 = f1_score(y_pred = y_pred,y_true = yvalid)
print(CM)
print(f1)

0.999
0.359
[[1079  184]
 [ 234  787]]
0.7901606425702811


In [161]:
idd = test['id']
predictions = model.predict([xtest_text,xtest_location,xtest_keyword])

In [13]:
### pred for submission ###
idd = test['id']
predict = model.predict([xtest_text,xtest_location,xtest_keyword])
predictions = apply_proba_threshold(predict, thresh)

In [14]:
### save data ####
path = 'Submission/'
file_name = 'SubmissionV0.csv'
df = pd.DataFrame(idd)
df['target'] = predictions
print(df)
df.to_csv(path+file_name,index=False)

         id  target
0         0       1
1         2       1
2         3       1
3         9       1
4        11       1
...     ...     ...
3258  10861       1
3259  10865       1
3260  10868       1
3261  10874       1
3262  10875       1

[3263 rows x 2 columns]
